# Spanish Translation AB Test

## Import packages and read data

In [61]:
#--- import packages
import pandas as pd
import pandas_profiling
import os
from scipy.stats import ttest_ind
from sklearn import tree
import plotly
import graphviz
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [3]:
#--- read data
PATH = "/home/billzhan/OneDrive/Academic/Open/Lai/take home challenge/spanish_translation"
test_data = pd.read_csv(os.path.join(PATH,"test_table.csv"))
user_data = pd.read_csv(os.path.join(PATH,"user_table.csv"))

In [4]:
#--- check unique users

# test data
test_data['user_id'].nunique() == len(test_data['user_id'])

True

In [5]:
# user data
user_data['user_id'].nunique() == len(user_data['user_id'])

True

In [6]:
# Are they matched?
test_data['user_id'].size - user_data['user_id'].size

454

Looks like the user table is busted and we have some user ids missing. When joining, we have to be
careful to do not lose the user ids in the test table, but not in the user table.

In [7]:
#--- merge two data
data = pd.merge(test_data, user_data, on='user_id', how='left')
data['date'] = pd.to_datetime(data['date'])
pandas_profiling.ProfileReport(data)

Number of variables,12
Number of observations,453321
Total Missing (%),5.0%
Total size in memory,45.0 MiB
Average record size in memory,104.0 B
Numeric,2
Categorical,7
Boolean,2
Date,1
Text (Unique),0
Rejected,0


### First check whether Spanish convert better than others

In [8]:
#--- fill missing countries
data['country'] = data['country'].fillna('no_country')
#--- average conversion rate
data_conversion_country = data[data['test']==0].groupby(['country'],as_index=False)['conversion'].mean()
data_conversion_country.sort_values(by=['conversion'],ascending=False).head()

,country,conversion
14,Spain,0.079719
17,no_country,0.077551
6,El Salvador,0.053554
10,Nicaragua,0.052647
4,Costa Rica,0.052256


### Do t test to see if the two groups (no spanish) has difference

In [9]:
#--- split data set
data_t = data[data['country']!='Spain']

#--- t test
data_test = data_t.loc[data_t['test']==1,'conversion']
data_control = data_t.loc[data_t['test']==0,'conversion']
ttest_ind(data_test,data_control)

Ttest_indResult(statistic=-7.422512095954769, pvalue=1.15144687852198e-13)

In [10]:
#--- mean of two sets
print(data_test.mean(),data_control.mean())

0.043424713982118966 0.04833042316066309


Mmh…not in the test are converting at 4.8% while users in the test just at 4.3%. That’s a 10% drop,
which would be dramatic if it were true. The most likely reason for weird A/B test results are:
1. We didn’t collect enough data.
2. Some bias has been introduced in the experiment so that test/control people are not really
random

### Plot day by day the conversion rate

In [27]:
#--- data by day
data_t_by_day = data_t.groupby(['date','test'],as_index=False)['conversion'].mean()
data_t_by_day = data_t_by_day.pivot(index='date',columns='test',values='conversion')

In [40]:
data_t_by_day['ratio'] = data_t_by_day[1]/data_t_by_day[0]

In [49]:
data_t_by_day.reset_index(inplace=True)

In [57]:
#--- make plot
#--- visualization
test_vs_control_bydate_toplot = [go.Scatter(
            x = data_t_by_day['date'],
            y = data_t_by_day['ratio']
    )]
bydate_layout = go.Layout(
    title='<b>Test Control conversion rate by date</b>',
    xaxis=dict(title='Date'),
    yaxis=dict(title='test_vs_control')
)
#--- make plot
test_vs_control = go.Figure(data=test_vs_control_bydate_toplot, layout=bydate_layout)
plotly.offline.iplot(test_vs_control)

From the plot, we notice a couple of things:
1. Test has constantly been worse than control and there is relatively little variance across days. That
probably means that we do have enough data, but there was some bias in the experiment set up.
2. On a side note, we just ran it for 5 days. We should always run the test for at least 1 full week to
capture weekly patterns, 2 weeks would be much better

Time to find out the bias! Likely, there is for some reason some segment of users more likely to end up in
test or in control, this segment had a significantly above/below conversion rate and this affected the overall
results.

In an ideal world, the distribution of people in test and control for each segment should be the same. There
are many ways to check this. One way is to build a decision tree where the variables are the user
dimensions and the outcome variable is whether the user is in test or control. If the tree splits, it means 
that for given values of that variable you are more likely to end up in test or control. But this should be 
impossible! Therefore, if the randomization worked, the tree should not split at all (or at least not be able to 
separate the two classes well.

### Build decision tree to check split

In [80]:
data_t['date'] = data_t['date'].astype('object')
cate_cols = [col for col in data_t.columns if data_t[col].dtype==object]
cate_cols

/home/billzhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



['date',
 'source',
 'device',
 'browser_language',
 'ads_channel',
 'browser',
 'sex',
 'country']

In [81]:
one_hot_data = pd.get_dummies(data_t[cate_cols])
one_hot_data

,date_1448841600000000000,date_1448928000000000000,date_1449014400000000000,date_1449100800000000000,date_1449187200000000000,source_Ads,source_Direct,source_SEO,device_Mobile,device_Web,...,country_Guatemala,country_Honduras,country_Mexico,country_Nicaragua,country_Panama,country_Paraguay,country_Peru,country_Uruguay,country_Venezuela,country_no_country
1,0,0,0,0,1,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
5,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,1,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0
8,0,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,1,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
10,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
13,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [82]:
clf = tree.DecisionTreeClassifier()
data_tree = pd.concat([data_t[data_t.columns.difference(cate_cols)],one_hot_data],axis=1)

clf = clf.fit(data_tree[data_tree.columns.difference(['conversion','test'])], data_tree['test'])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').